# Make Metadata for the Mock tests

In [11]:
from   time import time
import matplotlib.pyplot as plt
import healpy as hp
import pymangle as pm
import numpy as np
import pandas as pd
import fitsio as ft
from   utils import hpixsum

## Generate Randoms on the Mask

In [8]:
def generate_mocks(path2ply, path2ran, path2ranhp, path2frac):
    pm_mask = pm.Mangle(path2ply)
    area    = pm_mask.get_area()
    Nran_tot = int(2500 * area)   # generate 2500/sq deg * 5158 sq. deg
    Nranexp  = Nran_tot * (hp.nside2pixarea(256, degrees=True)/area)
    
    print('total number of randoms %d on area %.6f'%(Nran_tot, area))
    print('Expected number of randoms per pixel %.6f'%Nranexp)

    t1 = time()
    ra_ran, dec_ran = pm_mask.genrand(Nran_tot)
    print('generate %d randoms within %.1f sec'%(ra_ran.size, time()-t1))
    randoms = {'ra':ra_ran.astype(np.float64), 'dec':dec_ran.astype(np.float64)}
    RANDOMS = pd.DataFrame(data=randoms)
    RANDOMS.to_hdf(path2ran,'randoms_radec', mode='w', format='fixed')    
    
    hpran = hpixsum(256, RANDOMS.ra.values, RANDOMS.dec.values)
    kw = dict(dtype=np.float64, fits_IDL=False, overwrite=True)
    hp.write_map(path2ranhp, hpran, **kw)
    #
    #
    fracgood = hpran / Nranexp.astype('f8')
    kw = dict(dtype=np.float64, fits_IDL=False, overwrite=True)
    hp.write_map(path2frac, fracgood, **kw)
    print('DONE!')
    
    
    
path2ran   = '/Volumes/TimeMachine/data/mocks/randoms.h5'
path2ply   = '/Volumes/TimeMachine/data/mocks/mask_ngc.ply'
path2ranhp = '/Volumes/TimeMachine/data/mocks/randoms.hp256.fits'
path2frac  = '/Volumes/TimeMachine/data/mocks/fracgood.hp256.fits'
generate_mocks(path2ply, path2ran, path2ranhp, path2frac)

total number of randoms 12892898 on area 5157.159494
Expected number of randoms per pixel 131.139625
generate 12892898 randoms within 34.1 sec
DONE!


In [9]:
ls -lt /Volumes/TimeMachine/data/mocks/

total 635072
-rw-r--r--  1 rezaie  staff    6298560 Feb 22 16:05 fracgood.hp256.fits
-rw-r--r--  1 rezaie  staff    6298560 Feb 22 16:05 randoms.hp256.fits
-rw-r--r--  1 rezaie  staff  309436792 Feb 22 16:05 randoms.h5
-rw-r--r--  1 rezaie  staff      81000 Feb 22 16:03 fftlog.out
-rw-r--r--  1 rezaie  staff    3023247 Feb 22 16:02 mask_ngc.ply


## Project mocks on Data footprint

In [13]:
def project_mock_data(dataname, fracname, mocktab, mockmsk, fraccut):    
    # read data
    data,h = ft.read(dataname, header=True)
    print('data size', data.size)

    # mock fracmap
    frac  = hp.read_map(fracname, verbose=False)
    mhpix = np.argwhere(frac>fraccut).flatten()
    print('mock > 0.0 ', mhpix.size)

    # find the overlap
    mockondata = np.in1d(data['hpind'], mhpix)
    datamock   = data[mockondata]
    datamock['fracgood'] = frac[datamock['hpind']]  # replace the fracgood with the mocks 
    datamock['label'][:] = 0.0

    print('Mock with imaging attrs ', datamock.size)
    h['Note'] = 'This is for the mocks'
    ft.write(mocktab, datamock, header=h, clobber=True)


    # make mask
    mask = np.zeros(12*256*256)
    mask[datamock['hpind']] = 1.0
    hp.write_map(mockmsk, mask, fits_IDL=False, overwrite=True)
    print('DONE!!!')

#
# data
dataname = '/Volumes/TimeMachine/data/DR7/eBOSS.ELG.NGC.DR7.table.fits'
fracname = '/Volumes/TimeMachine/data/mocks/fracgood.hp256.fits'
mocktab  = '/Volumes/TimeMachine/data/mocks/mocks.DR7.table.fits'
mockmsk  = '/Volumes/TimeMachine/data/mocks/mask.hp.256.fits'
fraccut  = 0.2
project_mock_data(dataname, fracname, mocktab, mockmsk, fraccut)

data size 187257
mock > 0.0  98734
Mock with imaging attrs  89672
DONE!!!


In [14]:
ls -lt /Volumes/TimeMachine/data/mocks/

total 670672
-rw-r--r--  1 rezaie  staff    3153600 Feb 22 16:55 mask.hp.256.fits
-rw-r--r--  1 rezaie  staff   15071040 Feb 22 16:55 mocks.DR7.table.fits
-rw-r--r--  1 rezaie  staff    6298560 Feb 22 16:05 fracgood.hp256.fits
-rw-r--r--  1 rezaie  staff    6298560 Feb 22 16:05 randoms.hp256.fits
-rw-r--r--  1 rezaie  staff  309436792 Feb 22 16:05 randoms.h5
-rw-r--r--  1 rezaie  staff      81000 Feb 22 16:03 fftlog.out
-rw-r--r--  1 rezaie  staff    3023247 Feb 22 16:02 mask_ngc.ply


In [15]:
from utils import hpix2radec

In [ ]:
mradec = hpix2radec(256, hpix)
